# Accessing and analysing web resources with pygplates

### Overview

This notebook goes through the steps of
- accessing geoscientific data directly from web servers and loading into python
- converting these data into a GPlates-compatible version
- carrying out spatial analysis that compares fossil occurences to the locations of interpreted paleoshorelines through geological time


### Part 1

The first part of this exercise goes through the actions of getting data sets directly from web services, and loading them into your python workspace. 

Let's start by doing something that has nothing to do with GPlates whatsoever. We use a python module called 'requests' to make a request to a web service run by the USGS, and load the data into a pandas dataframe (the function 'StringIO' is also used to help pandas recognise the data stream that comes back from the USGS web service as something resembling a csv file).

Note that when you run the next cell, you will have to wait for a while as the data is downloaded.

To quickly see what is in returned data, we can print the resulting text, which reveals a stream of data in csv format.


In [ ]:
import requests
from io import StringIO
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

# make the web request. The result will be in the object 'r'
r = requests.get('http://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&endtime=2017-01-01T00:00:00.000Z&minsig=500&starttime=2016-01-01T00:00:00.000Z')

print r.text


The returned information will be more useful if we store it in a structured dataframe with column names:

In [ ]:
# read the returned data into a pandas dataframe
df = pd.read_csv(StringIO(r.text))

print df

Plotting the data shows us a bit more about what we have downloaded

In [ ]:
plt.scatter(df.longitude,df.latitude,c=df.mag,edgecolor='')
plt.colorbar()
plt.show()


### Questions:

- What exactly did we request, and what are we plotting?
- How can we vary the request to get a different subset of the data within the catalogue we are accessing
- Can you write a loop that creates a time-series of images with data for different time periods?


### Fossils

Go to this site:

https://paleobiodb.org/navigator/

As well as playing around with the interface, you should be able to find a way to access the data via http requests just like the one above. In fact, you can use the pbdb navigator to select which type of subset of the database (fossil type, geological period) you want to download. The navigator will provide you with a url that describes the request including the selection criteria you set.

Try to load a subset of the fossil data into a pandas dataframe, following a similar recipe to that described above

Note, there is at least one non-obvious trick here, which is that you need to specify the output format that you want the data to be sent to you in. You'll need to set this in the http request string.  
[HINT: the default is 'json', buit you don't want that.]

Download some data and figure out which columns contain what information.  
[you can see the column names for any dataframe by running 'df.columns']


#### Creating a GPlates Feature Collection

To do something with these fossil data - either with GPlates or pygplates - we first need to convert the data into a 'feature collection'. The following example shows how to take the coordinates from each fossil occurence in the dataframe and make a GPlates 'Point feature'. We append each feature to a big list of features, then save this to a 'Feature Collection' that GPlates can read.

In [ ]:
import pygplates

fossil_point_features = []
for index,row in df.iterrows():
    point = pygplates.PointOnSphere(float(row.lat),float(row.lng))
    point_feature = pygplates.Feature()
    point_feature.set_geometry(point)
    fossil_point_features.append(point_feature)
    
# Having created a list of features, we need to tell pygplates that this is a coherent 'Feature Collection'
# Then, we can save it to a file
# [note that the file format is controlled by the file extension we enter - can be *.gpml, *.gmt or *.shp]
output_feature_collection = pygplates.FeatureCollection(fossil_point_features)
output_feature_collection.write('my_fossils.gpml')


Load this feature collection into GPlates and look at it.

Questions:
- what information is missing from this feature collection that would be useful to assign to it?
- how can we assign that information (either in GPlates or within the python notebook)?  

HINTS: 
- one of the tutorials should help on this point
- the set_shapefile_attribute('')


### Part 2: Fossils and depositional environments

The rest of this notebook begins to consider the question 'what do fossils tell us about paleoenvironments?'

First, look at what types of attributes we have for each fossil, by listing the columns


In [ ]:
df.columns

We are going to focus on the column called 'environment', which contains information about the sedimentary depositional environment of the rock that hosts the fossil (at least according to the interpretation of the geologist who found the fossil). Let's look at the values in that column (only the first part of the full dataframe is listed)

In [ ]:
df.environment

A useful method that can be applied to pandas dataframes is 'unique' - this lists all the different unique types of enivironment that are found in the 'environment' column. 

In [ ]:
df.environment.unique()


when we load data into a pandas dataframe object, we have access to a wide variety of methods that can operate on the data. For example, suppose we want to find all the rows in the dataframe where the 'environment' field contains the string 'marine' somewhere within the environment description: 

In [ ]:
df.environment.str.contains('marine')

In the above, we get back a list of boolean results that are True where the environment string contains marine, False where it does not (And NaN, Not-a-Number, where the original field was also NaN)

We can use this result as an index to create a new, smaller dataframe that only contains the rows where the criteria we set are satisfied. Once we've made the subset dataframe, we can print the unique values in the environment column just for that dataframe, to check that our selection has worked. 

In [ ]:
subset = df.where(df.environment.str.contains('marine'))

print subset.environment.unique()

To see the spatial distribution of the data we have selected, we can plot a quick map 

In [ ]:
plt.plot(df.lng,df.lat,'b.')
plt.plot(subset.lng,subset.lat,'ro')
plt.show()


#### Questions

- what happens when you try a different search string? For example 'reef', or 'offshore'? Try composing a plot where you plot all the data points, but with different colours assigned to different environment types.
- how will you precisely formulate a test of consistency between paleoshorelines and the fossil points?
